<a href="https://colab.research.google.com/github/cassales/Hackathon2024Bootcamp/blob/master/Pandas_TS_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas tutorial

### In the following block we are going to import some standard libraries used for reading data and visualizing them.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Here we are going to download the csv file we have prepared, this file contains the levels of the river stage sensors and the rain gauges in the Coromandel region. We can run bash commands using by prepending the line with a '!'.

You can get a copy of the file here @ https://drive.google.com/file/d/1IsIl5465JevH2WJVasEhWDVxk_40vm8L/view?usp=sharing and upload it manually

In [ ]:
!wget https://raw.githubusercontent.com/nlim-uow/my_notes/main/river_rain.csv

### In the next block we are going to get the contents of the csv file and store that as our variable "myDataFrame"

In [ ]:
myDataFrame=pd.read_csv('river_rain.csv')

#### Here we are going to look into the headers and obtain some information regarding myDataFrame, there are a few things we should note here.



In [ ]:
myDataFrame.info()

a) Note the number of entries after the Index. Here it shows that we have 259449 entries, you will note that, the column Pinnacles Rain contains missing entries and has less than 259449 entries.

b) the other thing to note here is the Data Type of each of the columns, here the columns dd/MM/yyyy and hh:mm:ss is read in as "object" (that is to say they are read in as strings). So Pandas has no notion that they contain the date and time of the entries. This is something we will resolve later, when we get to the advanced section of this tutorial

c) The other four columns should be numeric (float64). You will notice that Opitonui is read in as an "object" (text). This is usually caused by one of the entries in the column containing text instead of numerical values. (in this case we intentionally coded a "not available" value as "N.A." which causes pandas to think that this column contains texts)

d) Note here that the info didn't show any issues with "CR Rain", however the missing values in "CR Rain" is coded as -99. While info() is a very helpful tool, it may not reveal all the problem with the data.

### Resolving missing entries
In the next block we are going to code the all text entries in the column Opitonui as missing values. This calls an in-built function in pandas that retains all numeric values in the column and convert all non-numeric values into "not available" (i.e. "missing value")


In [ ]:
myDataFrame['CR Rain'] = pd.to_numeric(myDataFrame['CR Rain'], errors='coerce')

It is usually a good practice to look at the info again after converting, to ensure that:

a) the conversion is done correctly

b) not too many entries are dropped in the conversion process

In [ ]:
myDataFrame.info()

Another good practice to do is to quickly visualize the data and obtain some summary statistics, we will go deeper into this later. This will reveal that the person supplying the data used -99 to code "missing" in the Opitonui column.


In [ ]:
myDataFrame.plot.box()
plt.show()
myDataFrame.describe()

Notice that the boxplot and the min value for "Opitonui" is contains an impossible value '-99'. We will now resolve this by replacing all occurance of -99 with "missing" (note here that np.nan is a special variable for not a number/missing)

In [ ]:
myDataFrame['Opitonui']=myDataFrame['Opitonui'].replace(-99,np.nan)

Again, it is a good practice to visualize the data and obtain some summary statistics before proceeding

In [ ]:
myDataFrame.plot.box()
plt.show()
myDataFrame.describe()

Now we are ready to deal with the missing values. There are some common methods to handle missing values

a) drop the entries with missing values

b) fill the missing values with the mean/median of the columns

c) fill the missing values with the preceding values

d) apply interpolation to fill in the missing values (will be covered in Advanced Pandas)

Droping entries with missing values is straight forward, dropna() to create a new dataframe that has the missing values dropped. Notice that the new dataframe is smaller and only contains the entries where the rows are complete.

In [ ]:
myDataFrame_dropping_na=myDataFrame.dropna()
myDataFrame_dropping_na.info()
myDataFrame_dropping_na.describe()

We can use the function fillna() to fill in the values for the missing entries with another value, in the first block we are replacing the missing value with the median of the column, and in the second block we are replacing the missing value with the mean of the column.

In [ ]:
# fill missing values with median for numeric columns only
numeric_columns = myDataFrame.select_dtypes(include=['number']).columns
myDataFrame_fill_median = myDataFrame.fillna(myDataFrame[numeric_columns].median())
myDataFrame_fill_median.info()
myDataFrame_fill_median.describe()

In [ ]:
myDataFrame_fill_median.values.shape
myDataFrame_fill_median['Tairua']+(np.random.randn(len(myDataFrame_fill_median))*myDataFrame_fill_median['Tairua'].std()*0.1)

In [ ]:
myDataFrame_fill_mean=myDataFrame.fillna(myDataFrame[numeric_columns].mean())
myDataFrame_fill_mean.info()
myDataFrame_fill_mean.describe()

We can also use the argument 'method="ffill"' to fill the missing value with the preceding values. Note that if the first entry contains missing values, the resulting dataframe will still contain missing value.

In [ ]:
myDataFrame_fill_forward=myDataFrame.fillna(method='ffill')
myDataFrame_fill_forward.info()
myDataFrame_fill_forward.describe()

For the fourth method, we will come back to that later when we get into the more advanced topic, as it would require providing pandas/python with a bit more information before we are able to do that.

### Data visualization and summarization
Pandas has some in-built visualization and we can use either Seaborn or any of the other plotting libraries to supplement the inbuilt visualization. If you have multiple plot commands in the same plot and you want a new figure for each plot, it is usually good practice to end every figure with plt.show()

Plotting a kernel density plot (histogram) of the data. You can either combine them within the same figure, or have each column individually

In [ ]:
myDataFrame.plot.kde()
plt.show()
myDataFrame['CR Rain'].plot.kde()
plt.show()
myDataFrame['Pinnacles Rain'].plot.kde()
plt.show()

If you prefer bins, we can plot this alternate histogram of the data. You can either combine them within the same figure, or have each column individually

In [ ]:
myDataFrame.plot.hist(bins=100)
plt.show()
myDataFrame['Tairua'].plot.hist(bins=30)
plt.show()
myDataFrame['Opitonui'].plot.hist(bins=30)
plt.show()

We can also visualize the data sequentially as a line plot




In [ ]:
myDataFrame.plot.line()
plt.show()
myDataFrame['Tairua'].plot.line()
plt.show()

Or visualize the relationship between two columns

In [ ]:
myDataFrame.plot.scatter('Pinnacles Rain','CR Rain')
plt.show()

The in-built pandas methods are quite powerful, however it may be too limited for some users, here we will look at another library called seaborn that has more plots

Here we see a pairwise scatter plot using the library seaborn, you can look at the documentations for additional control as to what goes to the different axes, or to add additional annotations on to the plot

In [ ]:
sns.pairplot(data=myDataFrame)
plt.show()

In [ ]:
sns.scatterplot(data=myDataFrame, x='Tairua',y='Opitonui', size='CR Rain')
plt.show()

In [ ]:
sns.violinplot(x=myDataFrame['Tairua'])
plt.show()

Other python plotting libraries are altair/plotly/bokeh, however these libraries do not work well with colab, and may crash if you are dealing with large datasets, I will show an example with the aggregated / filtered data in a later example

In [ ]:
#import altair as alt
#alt.Chart(myDataFrame).mark_point().encode(x='Pinnacles Rain',y='CR Rain')


### Data accessing and manipulations

We can concatenate text (objects) using the + operator, here we are showing what happens when we concatenate the first two columns together and joining them with a space.

In [ ]:
display(myDataFrame['dd/MM/yyyy']+" "+myDataFrame['hh:mm:ss'])

We can also do any numerical operation we want to the numerical columns. Note here that if the operation is mathematically invalid (i/e divide by 0), it would return NaN and would be treated as "missing". Also note that any operation with a missing value is automatically treated as invalid, which is why it may make sense to do the operation after you have resolved the missing values

In [ ]:
print('Tairua + Pinnacles')
display(myDataFrame_fill_median['Tairua']+myDataFrame_fill_median['Pinnacles Rain'])
print('Tairua/Opitonui')
display(myDataFrame_fill_median['Tairua']/myDataFrame_fill_median['Opitonui'])

#### Creating new columns and assigning values based on other columns
Creating new columns programmatically is very straight forward in Pandas/Python. All we have to do is to assign the outputs to the dataframe and the corresponding column name.

In [ ]:
myDataFrame_fill_median['River Ratio']=myDataFrame_fill_median['Tairua']/myDataFrame_fill_median['Opitonui']
myDataFrame_fill_median['Total Rain']=myDataFrame_fill_median['CR Rain']+myDataFrame_fill_median['Pinnacles Rain']
display(myDataFrame_fill_median.describe())

myDataFrame_fill_median['River Ratio'].plot.kde()


We have already looked at methods to obtain the statistics, note that we are not limited to just the standard summary statistics, pandas support more statistics than that

In [ ]:
print("The average for each column:")
display(myDataFrame_fill_median.mean(numeric_only=True))
print("")
print("The variance for each column:")
display(myDataFrame_fill_median.var(numeric_only=True))

print("")
print("Table of the summary statistics in each column:")
display(myDataFrame_fill_median.describe())

print("")
print("These are some of the additional statistics supported:")
display(myDataFrame_fill_median[numeric_columns].agg(['sum','min','max','mean','median','var','std','kurt','skew','nunique']))

We can address contents in the data frame by location and by value. For instance, if we want to find out what is the river level of Tairua at the start of the dataset, we can access it by using the following
`myDataFrame.loc[0,'Tairua']`.

Note that in python we use zero based index meaning the first entry starts with 0. You can also access the values using the index of the column myDataFrame.iloc[0,2]. Personally I prefer iloc, as it is more consistent and functions better when the columns are not properlly named

In [ ]:
myDataFrame_fill_median.info()

In [ ]:
print(myDataFrame_fill_median.loc[0,'Tairua'])
print(myDataFrame_fill_median.iloc[0,2])


You can also display a range of values. The following commands display, respectively: the first 5 entries, the last 5 entries, and the 100th to 110th entries for the Tairua river. Notice that the index is **not**
 "right"-inclusive

In [ ]:
print('1 - First 5 entries\n', myDataFrame_fill_median.iloc[0:5,2])
print('\n2  - Last 5 entries\n', myDataFrame_fill_median.iloc[-5:,2])
print('\n3 - 100th to 110th entries\n', myDataFrame_fill_median.iloc[100:110,2])

Obtaining the values for all columns in entries 100-109

In [ ]:
display(myDataFrame_fill_median.iloc[100:110,:])

Obtaining values in a column

In [ ]:
print(myDataFrame_fill_median['Tairua'])

In [ ]:
display(myDataFrame_fill_median[['Tairua','Opitonui']])

#### Accessing / subsetting dataframes by values.
We can also access specific rows of the dataframe by conditions in the dataframe. In the following example we are filtering the dataset to just when the "CR rain" is greater than 0, storing that as myDataFrameFiltered. and we will then visualize the river levels

In [ ]:
myDataFrame_filtered=myDataFrame_fill_median[myDataFrame_fill_median['CR Rain']>0]
display(myDataFrame_filtered.describe())
myDataFrame_filtered['Tairua'].plot.hist(bins=30)

## Advanced pandas methods
### Converting columns to datetime type
The following command tells pandas how to parse the text and understand it as a date time. Here we are concatenating the date column and the time column, and then converting it to a datetime this gives pandas a notion of time in the data

In [ ]:
myDataFrame['datetime_unparsed']=myDataFrame['dd/MM/yyyy']+" "+myDataFrame['hh:mm:ss']
myDataFrame['datetime_parsed']=pd.to_datetime(myDataFrame['datetime_unparsed'],format='%d/%m/%Y %H:%M:%S')
myDataFrame.info()

It is usually helpful to set the datetime format eventhough Pandas can usually infer the format, mainly because Pandas tend to use the US format of month-day-year if there are not enough non US dates

In [ ]:
myDataFrame_datetime=myDataFrame.set_index('datetime_parsed')
display(myDataFrame_datetime)


(Optional) Let's drop the redundant columns as it helps with processing the data

In [ ]:
myDataFrame_datetime=myDataFrame_datetime.drop(columns=['dd/MM/yyyy','hh:mm:ss','datetime_unparsed'])
myDataFrame_datetime.info()
myDataFrame_datetime.describe()

Now that pandas "understand" the data as a time series, we are able to do more advanced features with the data. As I've alluded earlier, let's try filling the NA values by interpolating the values.

In [ ]:
myDataFrame_datetime_fill_interpolate=myDataFrame_datetime.interpolate(method='time')

### Creating rolling sums
One of the useful features in pandas to create a rolling sum, i.e. to get the total rain over a period of time. Note that you may use any aggregation function not neccessarily the sum().


In [ ]:
myDataFrame_datetime_rolling_sum=myDataFrame_datetime_fill_interpolate
myDataFrame_datetime_rolling_sum['12 Hour CR Rain']=myDataFrame_datetime_fill_interpolate['CR Rain'].rolling('12H').sum()
myDataFrame_datetime_rolling_sum['24 Hour CR Rain']=myDataFrame_datetime_fill_interpolate['CR Rain'].rolling('24H').sum()
myDataFrame_datetime_rolling_sum['48 Hour CR Rain']=myDataFrame_datetime_fill_interpolate['CR Rain'].rolling('2D').sum()
myDataFrame_datetime_rolling_sum['96 Hour CR Rain']=myDataFrame_datetime_fill_interpolate['CR Rain'].rolling('4D').sum()

In [ ]:
sns.kdeplot(data=myDataFrame_datetime_rolling_sum, x='96 Hour CR Rain', fill=True)
plt.show()
display(myDataFrame_datetime_rolling_sum.describe())

Another thing we can do is to resample the data at a different frequency, in the example below we are resampling the data at a different sample rate. When we resample at a you may choose to use a single aggregating function, or choose to aggregate it differently by column.

In [ ]:
myDataFrame_1h_resampled=myDataFrame_datetime_fill_interpolate.resample('1H').agg({'Tairua': 'mean',
                             'Opitonui':'mean',
                             'CR Rain':'sum',
                             'Pinnacles Rain': 'sum'})
display(myDataFrame_1h_resampled)
myDataFrame_24h_resampled=myDataFrame_datetime_fill_interpolate.resample('24H').agg({'Tairua': 'mean',
                             'Opitonui':'mean',
                             'CR Rain':'sum',
                             'Pinnacles Rain': 'sum'})
display(myDataFrame_24h_resampled)

You may resample it at a higher frequency, as well, however notice what happens when you choose the same aggregating function as before.

In [ ]:
myDataFrame_2p5m_resampled=myDataFrame_datetime_fill_interpolate.resample('2.5T').agg({'Tairua': 'mean',
                             'Opitonui':'mean',
                             'CR Rain':'sum',
                             'Pinnacles Rain': 'sum'})
display(myDataFrame_2p5m_resampled)

In this case, it would make more sense to instead combine what we done previously and interpolate to obtain the inbetween values.

In [ ]:
myDataFrame_2p5m_resampled=myDataFrame_datetime_fill_interpolate.resample('2.5T').interpolate(method='time')
display(myDataFrame_2p5m_resampled)
myDataFrame_2p5m_resampled.info()

One very useful thing we can also do that may not appear to make sense at first is to resample at the same sampling rate as the original data. This is useful if you suspect that your data is missing entries. (Here we are missing about 33 entries)

In [ ]:
myDataFrame_5m_resampled=myDataFrame_datetime_fill_interpolate.resample('5T').interpolate(method='time')
display(myDataFrame_5m_resampled)
display(myDataFrame_5m_resampled.info())
display(myDataFrame_datetime_fill_interpolate.info())


### Group by, sub tables

In the next block I am going to create some new columns, here I am extracting the month, and hour from the index and store that as the a new column

In [ ]:
myDataFrame_1h_resampled['month']=myDataFrame_1h_resampled.index.month
myDataFrame_1h_resampled['hour']=myDataFrame_1h_resampled.index.hour
myDataFrame_1h_resampled.info()
myDataFrame_1h_resampled.describe()


In [ ]:
display(myDataFrame_1h_resampled.groupby('month').mean())
display(myDataFrame_1h_resampled.groupby('hour').mean())

In [ ]:
display(myDataFrame_1h_resampled.groupby('month').agg(['mean','std','min','max']))

In [ ]:
import altair as alt
alt.Chart(myDataFrame_1h_resampled[myDataFrame_1h_resampled['month']==3]).mark_point().encode(x='CR Rain',y='Tairua').interactive()

We can also shift the values in the data, in this example we want to create a new column called future Tairua, which shows what would the river stage be 1 hour in the future, this will come in handy when we want to use python to do machine learning to predict the river stage

In [ ]:
myDataFrame_1h_resampled['future Tairua']=myDataFrame_1h_resampled['Tairua'].shift(-1, freq='H')

The next plot shows us a scatter plot of the current river stage and the future river stage. Here we show that the two values are fairly correlated and it may be possible to obtain good results using simple machine learning methods

In [ ]:
myDataFrame_1h_resampled.plot.scatter('Tairua','future Tairua')
plt.show()

#### Combining data frames
We can use the join command to combine two dataframes, the keyword rsuffix tells pandas to add "_new" to any column. The join command automatically tries to get the entries that matches the index and append that to the output dataframe. Notice that not all the records from the rolling sum dataframe could be appended to the

In [ ]:
myDataFrame_1h_resampled_joined=myDataFrame_1h_resampled.join(myDataFrame_datetime_rolling_sum,rsuffix='_new')
myDataFrame_1h_resampled.info()
myDataFrame_datetime_rolling_sum.info()
myDataFrame_1h_resampled_joined.info()
display(myDataFrame_1h_resampled_joined)